In This homework You will take a data set and build a model from the data to predict the target and evaluate the preformance using cross validation and grid search. If you are building a classification model you should adress which measure of accuracy is important and incorperate that in your analysis. There are four datasets that have been provided. Information for each of them can be found here:

Climate failure (pop_failures_fixed.dat):

https://archive.ics.uci.edu/dataset/252/climate+model+simulation+crashes

Forest Fires (forestfires.csv)

https://archive.ics.uci.edu/dataset/162/forest+fires

Wine Quality (winequality-white.csv,whinequality-red.csv)

https://archive.ics.uci.edu/dataset/186/wine+quality

If you wnat to choose your own dataset you can use the following link:

https://archive.ics.uci.edu/datasets

You should first state the question you are trying to answer whith the model and then read in the data and make sure that the data is cleaned and ready for modeling. 

Next you may want to do some exploratory data analysis and convert and or scale any features you may want to use. You do not need to use all the fetures in the dataset, but you should have an explaination for any features that you drop.

Next build an appropriate model to answer your question. Determine how well your model can answer your question  using cross-validation and grid search.

Finally communicate your findings in a few paragraphs.

In [3]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

df_wine_red = pd.read_csv("winequality-red.csv", sep=';')
# df_wine_white=pd.read_csv("winequality-white.csv", sep=';')
# df_fires=pd.read_csv('forestfires.csv')
# df_pop=pd.read_csv('pop_failures_fixed.dat',sep=',')

In [8]:
# Display some data, statistics, and summary info
df_wine_red.describe()
print(df_wine_red.head())
print(df_wine_red.info())

# Visualize the target variable (quality) distribution
plt.figure(figsize=(8, 6))
sns.countplot(x=df_wine_red['quality'], palette='viridis')
plt.title('Distribution of Wine Quality', fontsize=16)
plt.xlabel('Quality', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

# Data Preprocessing
# Features and target split
target = df_wine_red['quality']  # Extract target
features = df_wine_red.drop('quality', axis=1)  # Extract features

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=7, stratify=target)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build model and tune hyperparameter using Grid Search
rf = RandomForestClassifier(random_state=7)

# Define hyperparameters to tune
grid_tune = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Grid Search with Cross-Validation (cv=5)
grid_search = GridSearchCV(estimator=rf, param_grid=grid_tune, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the model
# Predict on the test set
y_pred = best_model.predict(X_test_scaled)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(y_test.unique()), yticklabels=sorted(y_test.unique()))
plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('Actual', fontsize=12)
plt.show()

# Step 5: Cross-Validation Scores
cv_scores = cross_val_score(best_model, X_train_scaled, y_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.4f}")


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000
